In [1]:
import pandas as pd
import cobra
from cobra.io import read_sbml_model
from corda import CORDA
from corda import reaction_confidence
from cobra import Model, Reaction, Metabolite
from gprofiler import GProfiler
import numpy as np
import os

In [2]:
input_model = "/home/users/lzehetner/data/human1/human1.xml"
mod = cobra.io.read_sbml_model(input_model)

In [3]:
input_transcriptome = "/home/users/lzehetner/data/paper4_aav/HEK_Dietmair/GSE36094_processed_data_filtered_by_detection-pvalue.txt"
transcriptome = pd.read_csv(input_transcriptome, sep = "\t")

In [4]:
transcriptome

,ID_REF,Sample 1,Sample 2,Sample 3,Sample 4,Sample 5,Sample 6,Sample 7,Sample 8,Sample 9,...,Sample 15,Sample 16,Sample 17,Sample 18,Sample 19,Sample 20,Sample 21,Sample 22,Sample 23,Sample 24
0,ILMN_1796063,10.909568,10.686033,10.484470,10.580420,10.821422,10.667875,10.668543,10.657857,10.719120,...,10.596481,10.520208,10.625893,10.597385,10.538295,10.604912,10.533972,10.763537,10.773874,10.608606
1,ILMN_1811966,7.509697,7.657644,7.697281,7.639011,7.520340,7.779817,7.551836,7.622506,7.843433,...,7.583145,7.832166,7.449888,7.727502,7.499623,7.722397,7.450478,7.417768,7.532130,7.273337
2,ILMN_1793729,9.661065,9.461523,9.441575,9.344593,9.432135,9.592894,9.394488,9.453710,9.514681,...,9.747644,9.671438,9.725853,9.601156,9.608487,9.331044,9.712728,9.328784,9.612157,9.367705
3,ILMN_1682799,10.749738,10.804832,10.786887,10.788665,10.758936,10.857105,10.866369,10.888417,10.710554,...,10.896396,10.775869,10.874435,10.843286,10.763537,10.696173,10.606106,10.835450,10.738942,10.712946
4,ILMN_3305563,7.416676,7.321956,7.551842,7.406942,7.540410,7.559454,7.274985,7.393043,7.657635,...,7.360854,7.349248,7.507573,7.306957,7.498274,7.497078,7.517227,7.358917,7.665639,7.395817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,ILMN_1791388,10.670905,10.730608,10.690599,10.537213,10.745895,10.628369,10.572055,10.545107,10.806923,...,10.670905,10.727159,10.720283,10.646798,10.553275,10.578889,10.429728,10.546045,10.398527,10.448278
17832,ILMN_1754126,8.525712,8.671791,8.767685,8.525182,8.754860,8.565907,8.493229,8.818182,8.670027,...,8.653864,8.502985,8.628427,8.794829,8.486958,8.668961,8.577495,8.505847,8.700040,8.628084
17833,ILMN_1707464,8.666101,8.916163,9.091163,9.222082,9.132319,9.230268,9.061613,9.001067,9.265903,...,9.209716,8.861835,9.216765,8.960703,9.072636,9.199874,9.227783,8.994122,9.044997,9.123413
17834,ILMN_1785618,8.368973,8.307045,8.356303,8.447054,8.453019,8.505772,8.381081,8.329230,8.139021,...,8.391881,8.492079,8.271358,8.269878,8.375524,8.255512,8.340678,8.181179,8.080509,8.288850


In [5]:
Ilmn_ID = transcriptome['ID_REF'].tolist()

In [31]:
gp = GProfiler(return_dataframe=True)
converted = gp.convert(organism='hsapiens',
            query=Ilmn_ID,
            target_namespace='ENSG')

convert_dict = converted.set_index('incoming').to_dict()['converted']

replaced = transcriptome.replace({"ID_REF": convert_dict})

# df = df[df["team"].str.contains("Team 1") == False]
tr = replaced[replaced["ID_REF"].str.contains("None")==False]

In [32]:
tr

,ID_REF,Sample 1,Sample 2,Sample 3,Sample 4,Sample 5,Sample 6,Sample 7,Sample 8,Sample 9,...,Sample 15,Sample 16,Sample 17,Sample 18,Sample 19,Sample 20,Sample 21,Sample 22,Sample 23,Sample 24
0,ENSG00000166326,10.909568,10.686033,10.484470,10.580420,10.821422,10.667875,10.668543,10.657857,10.719120,...,10.596481,10.520208,10.625893,10.597385,10.538295,10.604912,10.533972,10.763537,10.773874,10.608606
1,ENSG00000214584,7.509697,7.657644,7.697281,7.639011,7.520340,7.779817,7.551836,7.622506,7.843433,...,7.583145,7.832166,7.449888,7.727502,7.499623,7.722397,7.450478,7.417768,7.532130,7.273337
2,ENSG00000167173,9.661065,9.461523,9.441575,9.344593,9.432135,9.592894,9.394488,9.453710,9.514681,...,9.747644,9.671438,9.725853,9.601156,9.608487,9.331044,9.712728,9.328784,9.612157,9.367705
3,ENSG00000138134,10.749738,10.804832,10.786887,10.788665,10.758936,10.857105,10.866369,10.888417,10.710554,...,10.896396,10.775869,10.874435,10.843286,10.763537,10.696173,10.606106,10.835450,10.738942,10.712946
5,ENSG00000187867,7.551702,7.527827,7.771663,7.436065,7.660663,7.658081,7.817061,7.926140,7.738751,...,7.456589,7.224724,7.546217,7.834886,7.414538,7.664187,7.760509,7.655134,8.109273,7.644048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,ENSG00000142409,10.670905,10.730608,10.690599,10.537213,10.745895,10.628369,10.572055,10.545107,10.806923,...,10.670905,10.727159,10.720283,10.646798,10.553275,10.578889,10.429728,10.546045,10.398527,10.448278
17832,ENSG00000189410,8.525712,8.671791,8.767685,8.525182,8.754860,8.565907,8.493229,8.818182,8.670027,...,8.653864,8.502985,8.628427,8.794829,8.486958,8.668961,8.577495,8.505847,8.700040,8.628084
17833,ENSG00000290851,8.666101,8.916163,9.091163,9.222082,9.132319,9.230268,9.061613,9.001067,9.265903,...,9.209716,8.861835,9.216765,8.960703,9.072636,9.199874,9.227783,8.994122,9.044997,9.123413
17834,ENSG00000183963,8.368973,8.307045,8.356303,8.447054,8.453019,8.505772,8.381081,8.329230,8.139021,...,8.391881,8.492079,8.271358,8.269878,8.375524,8.255512,8.340678,8.181179,8.080509,8.288850


In [33]:
tr.rename(columns={'ID_REF': 'gene'}, inplace=True)

/home/users/lzehetner/.conda/envs/HEK_human1/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [34]:
tr

,gene,Sample 1,Sample 2,Sample 3,Sample 4,Sample 5,Sample 6,Sample 7,Sample 8,Sample 9,...,Sample 15,Sample 16,Sample 17,Sample 18,Sample 19,Sample 20,Sample 21,Sample 22,Sample 23,Sample 24
0,ENSG00000166326,10.909568,10.686033,10.484470,10.580420,10.821422,10.667875,10.668543,10.657857,10.719120,...,10.596481,10.520208,10.625893,10.597385,10.538295,10.604912,10.533972,10.763537,10.773874,10.608606
1,ENSG00000214584,7.509697,7.657644,7.697281,7.639011,7.520340,7.779817,7.551836,7.622506,7.843433,...,7.583145,7.832166,7.449888,7.727502,7.499623,7.722397,7.450478,7.417768,7.532130,7.273337
2,ENSG00000167173,9.661065,9.461523,9.441575,9.344593,9.432135,9.592894,9.394488,9.453710,9.514681,...,9.747644,9.671438,9.725853,9.601156,9.608487,9.331044,9.712728,9.328784,9.612157,9.367705
3,ENSG00000138134,10.749738,10.804832,10.786887,10.788665,10.758936,10.857105,10.866369,10.888417,10.710554,...,10.896396,10.775869,10.874435,10.843286,10.763537,10.696173,10.606106,10.835450,10.738942,10.712946
5,ENSG00000187867,7.551702,7.527827,7.771663,7.436065,7.660663,7.658081,7.817061,7.926140,7.738751,...,7.456589,7.224724,7.546217,7.834886,7.414538,7.664187,7.760509,7.655134,8.109273,7.644048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,ENSG00000142409,10.670905,10.730608,10.690599,10.537213,10.745895,10.628369,10.572055,10.545107,10.806923,...,10.670905,10.727159,10.720283,10.646798,10.553275,10.578889,10.429728,10.546045,10.398527,10.448278
17832,ENSG00000189410,8.525712,8.671791,8.767685,8.525182,8.754860,8.565907,8.493229,8.818182,8.670027,...,8.653864,8.502985,8.628427,8.794829,8.486958,8.668961,8.577495,8.505847,8.700040,8.628084
17833,ENSG00000290851,8.666101,8.916163,9.091163,9.222082,9.132319,9.230268,9.061613,9.001067,9.265903,...,9.209716,8.861835,9.216765,8.960703,9.072636,9.199874,9.227783,8.994122,9.044997,9.123413
17834,ENSG00000183963,8.368973,8.307045,8.356303,8.447054,8.453019,8.505772,8.381081,8.329230,8.139021,...,8.391881,8.492079,8.271358,8.269878,8.375524,8.255512,8.340678,8.181179,8.080509,8.288850


In [10]:
#tr = tr.set_index(tr.columns[0])

In [35]:
np_ind = [0,6,7,8,9,10,11,18,19,20,21,22,23]
# Use list comprehension to select the columns by these indices
np_col = [tr.columns[i] for i in np_ind]
tr_np = tr[np_col]
col_names = ['gene', 'NP_TP2_1', 'NP_TP2_2', 'NP_TP2_3', 'NP_TP3_1', 'NP_TP3_2', 'NP_TP3_3', 'NP_TP1_1', 'NP_TP1_2', 'NP_TP1_3', 'NP_TP4_1', 'NP_TP4_2', 'NP_TP4_3']  # Add your actual new names here
tr_np.columns = col_names

In [37]:
tr = tr_np

In [38]:
tr

,gene,NP_TP2_1,NP_TP2_2,NP_TP2_3,NP_TP3_1,NP_TP3_2,NP_TP3_3,NP_TP1_1,NP_TP1_2,NP_TP1_3,NP_TP4_1,NP_TP4_2,NP_TP4_3
0,ENSG00000166326,10.667875,10.668543,10.657857,10.719120,10.588682,10.477634,10.597385,10.538295,10.604912,10.533972,10.763537,10.773874
1,ENSG00000214584,7.779817,7.551836,7.622506,7.843433,7.655350,7.424281,7.727502,7.499623,7.722397,7.450478,7.417768,7.532130
2,ENSG00000167173,9.592894,9.394488,9.453710,9.514681,9.411075,9.263489,9.601156,9.608487,9.331044,9.712728,9.328784,9.612157
3,ENSG00000138134,10.857105,10.866369,10.888417,10.710554,10.951981,10.864812,10.843286,10.763537,10.696173,10.606106,10.835450,10.738942
5,ENSG00000187867,7.658081,7.817061,7.926140,7.738751,7.423178,7.409969,7.834886,7.414538,7.664187,7.760509,7.655134,8.109273
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,ENSG00000142409,10.628369,10.572055,10.545107,10.806923,10.589259,10.465357,10.646798,10.553275,10.578889,10.429728,10.546045,10.398527
17832,ENSG00000189410,8.565907,8.493229,8.818182,8.670027,8.586712,8.741508,8.794829,8.486958,8.668961,8.577495,8.505847,8.700040
17833,ENSG00000290851,9.230268,9.061613,9.001067,9.265903,9.284920,9.311511,8.960703,9.072636,9.199874,9.227783,8.994122,9.044997
17834,ENSG00000183963,8.505772,8.381081,8.329230,8.139021,8.408365,8.511821,8.269878,8.375524,8.255512,8.340678,8.181179,8.080509


In [39]:
input_gpr = "/home/users/lzehetner/gpr_human1.csv"

In [40]:
gpr_ori = pd.read_csv(input_gpr, sep = ";")

In [41]:
def rxn_extraction(gene_list):
    gpr = gpr_ori.iloc[:, [0,2]]
    rxn_wo_genes = gpr[gpr.iloc[:, 1].isna()]
    gpr.dropna(inplace=True)
    gpr_and = gpr[gpr['Gene-reaction association'].str.contains('and')]
    gpr = gpr[~gpr.isin(gpr_and)].dropna()
    gpr_or = gpr[gpr['Gene-reaction association'].str.contains('or')]
    gpr_one = gpr[~gpr.isin(gpr_or)].dropna()
    
    tr_list = gene_list.values.tolist()
    
    dict_gpr_one = gpr_one.set_index("Rxn name")["Gene-reaction association"].to_dict()
    rxns_w_one = [key for (key,value) in dict_gpr_one.items() if value in tr_list]

    gpr_and_1 = gpr_and[gpr_and['Gene-reaction association'].str.contains("\(")]

    gpr_and_2 = gpr_and_1[gpr_and_1['Gene-reaction association'].str.contains("\) or")]
    x = gpr_and_2["Gene-reaction association"].str.split(' or ').tolist()
    y = gpr_and_2["Rxn name"].tolist()
    gpr_and_dict_2 = {y[i]: x[i] for i in range(len(y))}
    df = pd.DataFrame(list(gpr_and_dict_2.items()))
    df = df.rename(columns={1: 'Genes'})
    df = df.explode('Genes')
    df_and = df[df["Genes"].str.contains("\(")]
    df_and['Genes'] = df_and['Genes'].str.replace("\( ", "")
    df_and['Genes'] = df_and['Genes'].str.replace(" \)", "")
    df_or = df[df["Genes"].str.contains(r'^E')]

    x = gpr_or["Gene-reaction association"].str.split(' or ').tolist()
    y = gpr_or["Rxn name"].tolist()
    gpr_or_dict = {y[i]: x[i] for i in range(len(y))}
    df = pd.DataFrame(list(gpr_or_dict.items()))
    df = df.rename(columns={1: 'Genes'})
    df1 = df.explode('Genes')
    df = pd.concat([df1, df_or])
    rxns_w_iso = df[df["Genes"].isin(tr_list)]
    rxns_w_iso_unique = rxns_w_iso[0].unique().tolist()

    x = gpr_and[~gpr_and.isin(gpr_and_1)].dropna()
    x1 = x["Gene-reaction association"].str.split(' and ').tolist()
    y = x["Rxn name"].tolist()
    gpr_and_dict = {y[i]: x1[i] for i in range(len(y))}
    df = pd.DataFrame(list(gpr_and_dict.items()))
    df1 = df.rename(columns={1: 'Genes'})
    df = pd.concat([df1, df_and])
    df['new'] = df['Genes'].explode().isin(tr_list).groupby(level=0).any()
    z = df[df[["new"]].all(axis=1)]
    z = z.iloc[:, [0,1]]
    rxns_w_compl = z.iloc[:, 0].values.tolist()

    nested_rxns = ["MAR04137", "MAR07161", "MAR07162"]
    rxns = list(set(rxns_w_one + rxns_w_iso_unique + rxns_w_compl + nested_rxns))
    
    return rxns

In [42]:
min_media_comp = [
    "MAR09061",
    "MAR09034",
    "MAR09035",
    "MAR09036",
    "MAR09038",
    "MAR09039",
    "MAR09040",
    "MAR09041",
    "MAR09042",
    "MAR09043",
    "MAR09044",
    "MAR09045",
    "MAR09046",
    "MAR09047",
    "MAR09048",
    "MAR09062",
    "MAR09063",
    "MAR09064",
    "MAR09065",
    "MAR09066",
    "MAR09068",
    "MAR09069",
    "MAR09076",
    "MAR09146",
    "MAR09109",
    "MAR09143",
    "MAR09144",
    "MAR09147",
    "MAR09150",
    "MAR09151",
    "MAR09153",
    "MAR09158",
    "MAR09159",
    "MAR09167",
    "MAR09269",
    "MAR09072",
    "MAR09145",
    "MAR09070",
    "MAR09071",
    "MAR09067",
    "MAR11420",
    "MAR09135"
]

secr_comp = ["MAR09422", # Thymine
             "MAR09142", # Nicotinate
             "MAR09437", # Uracil
             "MAR09378", # Nicotinamide
             "MAR11428", # Xanthine
             "MAR09358", # Hypoxanthine
             "MAR09021", # 5-Methyladenosine
             "MAR09253", # Adenine
             "MAR09341", # Betaine
             "MAR09353", # Guanine
             "MAR09848", # Dimethylglycine
             "MAR09290", # Creatine
             "MAR09131", # Sarcosine
             "MAR10428", # 5-Aminoevulinic acid
             "MAR04849", # Citrulline
             "MAR09087", # Ornithine
             "MAR11400", # Fumarate
             "MAR09415", # Succinate
             "MAR09025", # Pyroglutamate / 5-oxoproline
             "MAR11404", # Malate
             "MAR09286", # Citrate
             "MAR09861", # Xanthosine
             "MAR10431", # Allantoine
             "MAR09285" # Cholesterole
            ]

In [ ]:
for column in tr.columns:
    if column != 'gene':
        
        genes = pd.DataFrame(tr[["gene", column]])
        genes['mean'] = genes.mean(axis=1)
        genes = genes.loc[genes["mean"] >= 0.2]

        q = genes.quantile([0.00, 0.10, 0.25, 0.50, 0.75, 0.90, 1.00])
        col = 'mean'
        a = genes[((genes[col]>=q[col][0.10]) & (genes[col]<=q[col][1.00]))]
        b = a.iloc[:, 0]
        ntile = b.to_frame()

        # ## Rxn Extraction

        rxns = pd.DataFrame(rxn_extraction(ntile["gene"]), columns = ['Rxns'])

        print("Rxns are extracted")
        print(len(rxns))

        for r in mod.exchanges:
            mod.reactions.get_by_id(r.id).lower_bound = 0.0

        for r in min_media_comp:
            mod.reactions.get_by_id(r).lower_bound = -1000.0

        mod.solver = 'cplex'

        conf = {}
        for r in mod.reactions: conf[r.id] = -1
        for r in rxns["Rxns"]: conf[r] = 3    
        a = mod.exchanges
        b = []
        for r in a:
            b.append(r.id)
        for r in b: conf[r] = -1

        for r in min_media_comp: conf[r] = 3
        for r in secr_comp: conf[r] = 3
        conf["MAR13082"] = 3 # Biomass
        conf["MAR10023"] = 3 # Biomass export
        conf["MAR10024"] = 3 # Biomass export
        conf["MAR07160"] = 3 # DNA Pool reaction
        conf["MAR13086"] = 3 # RNA Pool reaction
        conf["MAR10065"] = 3 # Cofactor Pool reaction
        conf["MAR09727"] = 3 # Glycogen Pool reaction
        conf["MAR10063"] = 3 # Lipid Pool reaction
        conf["MAR10064"] = 3 # Metabolite Pool reaction
        conf["MAR10062"] = 3 # Protein Pool reaction
        conf.pop("Rxns", None)

        print(f"Model reconstruction starts for {column}")

        model = CORDA(mod, conf)
        model.build()
        model_reconstr = model.cobra_model()

        model_reconstr.objective = "MAR13082"
        print(model_reconstr.optimize().objective_value)

        reactions = []
        for r in model_reconstr.reactions:
            reactions.append(r.id)

        # >25% reconstruction

        mod = cobra.io.read_sbml_model(input_model)    

        a = genes[((genes[col]>=q[col][0.25]) & (genes[col]<=q[col][1.00]))]
        b = a.iloc[:, 0]
        ntile = b.to_frame()

        rxns = pd.DataFrame(rxn_extraction(ntile["gene"]), columns = ['Rxns'])

        for r in mod.exchanges:
            mod.reactions.get_by_id(r.id).lower_bound = 0.0

        for r in min_media_comp:
            mod.reactions.get_by_id(r).lower_bound = -1000.0

        mod.solver = 'cplex'

        conf = {}
        for r in mod.reactions: conf[r.id] = -1
        for r in rxns["Rxns"]: conf[r] = 3    
        a = mod.exchanges
        b = []
        for r in a:
            b.append(r.id)
        for r in b: conf[r] = -1

        for r in min_media_comp: conf[r] = 3
        for r in secr_comp: conf[r] = 3
        conf["MAR13082"] = 3 # Biomass
        conf["MAR10023"] = 3 # Biomass export
        conf["MAR10024"] = 3 # Biomass export
        conf["MAR07160"] = 3 # DNA Pool reaction
        conf["MAR13086"] = 3 # RNA Pool reaction
        conf["MAR10065"] = 3 # Cofactor Pool reaction
        conf["MAR09727"] = 3 # Glycogen Pool reaction
        conf["MAR10063"] = 3 # Lipid Pool reaction
        conf["MAR10064"] = 3 # Metabolite Pool reaction
        conf["MAR10062"] = 3 # Protein Pool reaction
        conf.pop("Rxns", None)

        print(f"Model reconstruction starts for {column}")

        model = CORDA(mod, conf)
        model.build()
        model_reconstr = model.cobra_model()

        for r in model_reconstr.reactions:
            reactions.append(r.id)

        # # >50% reconstruction

        mod = cobra.io.read_sbml_model(input_model)    

        a = genes[((genes[col]>=q[col][0.50]) & (genes[col]<q[col][1.00]))]
        b = a.iloc[:, 0]
        ntile = b.to_frame()

        rxns = pd.DataFrame(rxn_extraction(ntile["gene"]), columns = ['Rxns'])

        for r in mod.exchanges:
            mod.reactions.get_by_id(r.id).lower_bound = 0.0

        for r in min_media_comp:
            mod.reactions.get_by_id(r).lower_bound = -1000.0

        mod.solver = 'cplex'

        conf = {}
        for r in mod.reactions: conf[r.id] = -1
        for r in rxns["Rxns"]: conf[r] = 3    
        a = mod.exchanges
        b = []
        for r in a:
            b.append(r.id)
        for r in b: conf[r] = -1

        for r in min_media_comp: conf[r] = 3
        for r in secr_comp: conf[r] = 3
        conf["MAR13082"] = 3 # Biomass
        conf["MAR10023"] = 3 # Biomass export
        conf["MAR10024"] = 3 # Biomass export
        conf["MAR07160"] = 3 # DNA Pool reaction
        conf["MAR13086"] = 3 # RNA Pool reaction
        conf["MAR10065"] = 3 # Cofactor Pool reaction
        conf["MAR09727"] = 3 # Glycogen Pool reaction
        conf["MAR10063"] = 3 # Lipid Pool reaction
        conf["MAR10064"] = 3 # Metabolite Pool reaction
        conf["MAR10062"] = 3 # Protein Pool reaction
        conf.pop("Rxns", None)

        print(f"Model reconstruction starts for {column}")

        model = CORDA(mod, conf)
        model.build()
        model_reconstr = model.cobra_model()

        for r in model_reconstr.reactions:
            reactions.append(r.id)

        # # >75% reconstruction

        mod = cobra.io.read_sbml_model(input_model)    

        a = genes[((genes[col]>=q[col][0.75]) & (genes[col]<q[col][1.00]))]
        b = a.iloc[:, 0]
        ntile = b.to_frame()

        rxns = pd.DataFrame(rxn_extraction(ntile["gene"]), columns = ['Rxns'])

        for r in mod.exchanges:
            mod.reactions.get_by_id(r.id).lower_bound = 0.0

        for r in min_media_comp:
            mod.reactions.get_by_id(r).lower_bound = -1000.0

        mod.solver = 'cplex'

        conf = {}
        for r in mod.reactions: conf[r.id] = -1
        for r in rxns["Rxns"]: conf[r] = 3    
        a = mod.exchanges
        b = []
        for r in a:
            b.append(r.id)
        for r in b: conf[r] = -1

        for r in min_media_comp: conf[r] = 3
        for r in secr_comp: conf[r] = 3
        conf["MAR13082"] = 3 # Biomass
        conf["MAR10023"] = 3 # Biomass export
        conf["MAR10024"] = 3 # Biomass export
        conf["MAR07160"] = 3 # DNA Pool reaction
        conf["MAR13086"] = 3 # RNA Pool reaction
        conf["MAR10065"] = 3 # Cofactor Pool reaction
        conf["MAR09727"] = 3 # Glycogen Pool reaction
        conf["MAR10063"] = 3 # Lipid Pool reaction
        conf["MAR10064"] = 3 # Metabolite Pool reaction
        conf["MAR10062"] = 3 # Protein Pool reaction
        conf.pop("Rxns", None)

        print(f"Model reconstruction starts for {column}")

        model = CORDA(mod, conf)
        model.build()
        model_reconstr = model.cobra_model()

        for r in model_reconstr.reactions:
            reactions.append(r.id)

        # # >90% reconstruction

        mod = cobra.io.read_sbml_model(input_model)    

        a = genes[((genes[col]>=q[col][0.90]) & (genes[col]<q[col][1.00]))]
        b = a.iloc[:, 0]
        ntile = b.to_frame()

        rxns = pd.DataFrame(rxn_extraction(ntile["gene"]), columns = ['Rxns'])

        for r in mod.exchanges:
            mod.reactions.get_by_id(r.id).lower_bound = 0.0

        for r in min_media_comp:
            mod.reactions.get_by_id(r).lower_bound = -1000.0

        mod.solver = 'cplex'

        conf = {}
        for r in mod.reactions: conf[r.id] = -1
        for r in rxns["Rxns"]: conf[r] = 3    
        a = mod.exchanges
        b = []
        for r in a:
            b.append(r.id)
        for r in b: conf[r] = -1

        for r in min_media_comp: conf[r] = 3
        for r in secr_comp: conf[r] = 3
        conf["MAR13082"] = 3 # Biomass
        conf["MAR10023"] = 3 # Biomass export
        conf["MAR10024"] = 3 # Biomass export
        conf["MAR07160"] = 3 # DNA Pool reaction
        conf["MAR13086"] = 3 # RNA Pool reaction
        conf["MAR10065"] = 3 # Cofactor Pool reaction
        conf["MAR09727"] = 3 # Glycogen Pool reaction
        conf["MAR10063"] = 3 # Lipid Pool reaction
        conf["MAR10064"] = 3 # Metabolite Pool reaction
        conf["MAR10062"] = 3 # Protein Pool reaction
        conf.pop("Rxns", None)

        print(f"Model reconstruction starts for {column}")

        model = CORDA(mod, conf)
        model.build()
        model_reconstr = model.cobra_model()

        for r in model_reconstr.reactions:
            reactions.append(r.id)

        # print("10% reconstruction led to:", len(reactions))
        
        reactions = list(set(reactions))

        reactions_df[column] = reactions_df['reaction_id'].apply(lambda x: 1 if x in reactions else 0)

#reactions_df = pd.DataFrame(reactions, columns = ['Rxns']).drop_duplicates()
# the export file needs also to be changed every time
reactions_df.to_csv(output_rxns, index=False, header=True)

/home/users/lzehetner/.conda/envs/HEK_human1/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """
/home/users/lzehetner/.conda/envs/HEK_human1/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/users/lzehetner/.conda/envs/HEK_human1/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
/home/users/lzehetner/.conda/envs/HEK_human1/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:

Rxns are extracted
5542
Model reconstruction starts for NP_TP2_1


/home/users/lzehetner/.conda/envs/HEK_human1/lib/python3.7/site-packages/cobra/core/group.py:107: UserWarning: need to pass in a list
  warn("need to pass in a list")


53.569984624900286
Model reconstruction starts for NP_TP2_1
